In [2]:
import os

# change the working directory to the location of files
os.chdir('D:/eo_data/Ibague/')
print(os.getcwd())
# store the files list to a variable
eo_files = os.listdir(os.getcwd())
print(eo_files)

D:\eo_data\Ibague
['data', 'S1_GRD', 'S2_L1C']


In [18]:
# Unzip files
import zipfile

# Check if a data folder exist
#if not os.path.exists('data'):
#    os.makedirs('data')
#    print 'data folder' + ' was created'
# Check if the name of the data folder is in
#if 'data' in eo_files:
#    eo_files.remove('data')
##TODO catch error when a directory is in eo_files

def unzipfiles(path):
    """
    Unzips every zipfile in the path, and stores in directory with zipfile name+.SAFE
    Args:
        path (string): string of directory where zipfiles are located 
    """
    for im_id in os.listdir(path):
        if not os.path.exists('data/'+im_id[:-3]+'SAFE'):
            print('Unzipping ' + im_id)
            zip_ref = zipfile.ZipFile(path+im_id, 'r')
            zip_ref.extractall('data')
            zip_ref.close()
        else:
            print(im_id[:-4] + ' was already uncompressed')

In [19]:
os.getcwd()
unzipfiles("D:/eo_data/Ibague/S2_L1C/")

S2A_MSIL1C_20171021T153111_N0205_R025_T18NVL_20171021T153112 was already uncompressed
S2A_MSIL1C_20171210T152631_N0206_R025_T18NVK_20171210T170838 was already uncompressed
S2A_MSIL1C_20171220T153111_N0206_R025_T18NVK_20171220T170223 was already uncompressed
S2A_MSIL1C_20171220T153111_N0206_R025_T18NVL_20171220T170223 was already uncompressed
S2B_MSIL1C_20171016T153059_N0205_R025_T18NVK_20171016T153058 was already uncompressed
S2B_MSIL1C_20171016T153059_N0205_R025_T18NVL_20171016T153058 was already uncompressed
S2B_MSIL1C_20171105T153059_N0206_R025_T18NVK_20171105T170109 was already uncompressed
S2B_MSIL1C_20171105T153059_N0206_R025_T18NVL_20171105T170109 was already uncompressed
S2B_MSIL1C_20171205T152629_N0206_R025_T18NVK_20171205T170944 was already uncompressed
S2B_MSIL1C_20171205T152629_N0206_R025_T18NVL_20171205T170944 was already uncompressed
S2B_MSIL1C_20171215T152629_N0206_R025_T18NVK_20171215T201906 was already uncompressed
S2B_MSIL1C_20171215T152629_N0206_R025_T18NVL_20171215T

## Process L1C to L2A images using sen2cor
Call in console. To-do: check integration
## Pre-process L2A images

In [1]:
# import required packages
import snappy
import re

from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from snappy import ProductIO
from snappy import HashMap
import shutil
import os  
import ast

from snappy import GPF

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = snappy.jpy.get_type('java.util.HashMap')

In [2]:
# set location of products
products_dir = "D:/eo_data/Ibague/data/"

# get geoJSON file with region extent
regPath = "//dapadfs/workspace_cluster_6/TRANSVERSAL_PROJECTS/MADR/COMPONENTE_2/Imagenes_Satelitales/Sentinel_2/" + \
                      "JSON_Ibague/IbagueJSON.geojson"
geom = geojson_to_wkt(read_geojson(regPath))

# get a list of S2 L2A products
prdlist = filter(re.compile(r'^S2.....L2A').search, os.listdir(products_dir))

## Create a dictionary to read Sentinel-1 L1 GRD products
product = {}
for element in prdlist:
    product[element[:-4]] = {}


product.keys()

['S2A_MSIL2A_20171210T152631_N0206_R025_T18NVK_20171210T170838.']

In [19]:
## iterate over S2 L2A products
    
for key, value in product.iteritems():    
    # Read the product
    value['GRD'] = ProductIO.readProduct(products_dir+key+'SAFE/MTD_MSIL2A.xml')
    
    # Resample all bands to 10m resolution
    resample_subset = HashMap()
    resample_subset.put('targetResolution', 10)
    print('Resampling...')
    value['res10'] = GPF.createProduct("Resample", resample_subset, value['GRD'])
    
    # Subset to area of interest
    param_subset = HashMap()
    param_subset.put('geoRegion', geom)
    param_subset.put('outputImageScaleInDb', False)
    param_subset.put('sourceBandNames', 'B2, B3, B4, B8, B11, B12, quality_cloud_confidence,quality_scene_classification')
    print('Subsetting...')
    value['sub'] = GPF.createProduct("Subset", param_subset, value['res10'])
    
    # Write product
    print('Writing...')
    ProductIO.writeProduct(value['sub'], products_dir+'pre/'+key+'subset', 'BEAM-DIMAP')
    
    # Dispose all the intermediate products
    value['GRD'].dispose()
    value['res10'].dispose()
    value['sub'].dispose()
    


Resampling...
Subsetting...
Writing...


In [4]:
from snappy import ProductIO, GPF, jpy

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = jpy.get_type('java.util.HashMap')

#collection of preprocessed files
s1 = 'D:/eo_data/Ibague/data/prep/VH_specklefil_dB.dim'
s2 = 'D:/eo_data/Ibague/data/prep/VV_specklefil_dB.dim'
master = 'D:/eo_data/Ibague/data/test.data/B1.img'

processed_files = [master,s1,s2]

product_set=[]

for f in processed_files:
    product_set.append(ProductIO.readProduct(f))
    print("Reading "+f)
      
#define the stack parameters
params = HashMap()
params.put('resamplingType', 'NEAREST_NEIGHBOUR')
params.put('initialOffsetMethod', 'Product Geolocation')
params.put('extent', 'Master')

#make the stack
print("Creating stack...")
create_stack = GPF.createProduct('CreateStack', params, product_set)

#write the stack
print("Saving resulting product...")
ProductIO.writeProduct(create_stack, 'D:/eo_data/Ibague/data/create_stack', 'BEAM-DIMAP')
print("Done.")

Reading D:/eo_data/Ibague/data/test.data/B1.img
Reading D:/eo_data/Ibague/data/prep/VH_specklefil_dB.dim
Reading D:/eo_data/Ibague/data/prep/VV_specklefil_dB.dim
Creating stack...
Saving resulting product...
Done.
